In [1]:
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
import time
import optuna
from sklearn.metrics import f1_score, classification_report
from machine_learning.combination1 import Combined_xgb_logreg
from machine_learning.SVM_defopt import SVMOptimizer

In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra**

In [3]:
close_data = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False)

In [4]:
xgboost_optimizer = XGBOptimizer(x_train, y_train, x_test, y_test)
xgboost_best_params, best_model_xgboost = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train, y_train, x_test, y_test)
reg_log_best_params, best_model_reg_log = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train, y_train, x_test, y_test)
SVC_best_params, best_model_SVC = SVC_optimizer.svm_optuna()
# xgboost_opt = XGBOptimizer(x_train, y_train, x_test, y_test).xgb_optuna()
# reg_log_opt = REGLOGOptimizer(x_train, y_train, x_test, y_test).reglog_optuna()
# combined_opt = Combined_xgb_logreg(x_train, y_train, x_test, y_test).combined_optuna()

[I 2024-03-08 17:11:38,620] A new study created in memory with name: no-name-633a70c1-de96-4c1d-8436-436a90858467
[I 2024-03-08 17:11:39,154] Trial 0 finished with value: 0.678879670360265 and parameters: {'n_estimators': 34, 'max_depth': 90, 'max_leaves': 32, 'learning_rate': 0.8092458606542622, 'booster': 'dart', 'gamma': 27.70865040715498, 'reg_alpha': 9.969288419086604, 'reg_lambda': 9.464427606469147}. Best is trial 0 with value: 0.678879670360265.
[I 2024-03-08 17:11:39,180] Trial 1 finished with value: 0.24755455229495862 and parameters: {'n_estimators': 42, 'max_depth': 96, 'max_leaves': 18, 'learning_rate': 1.9309101818843215, 'booster': 'gbtree', 'gamma': 16.943225198601933, 'reg_alpha': 0.9178077207306213, 'reg_lambda': 2.5978030539234127}. Best is trial 0 with value: 0.678879670360265.
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [17:11:39] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_d

F1: 0.678879670360265
Best hyperparameters: {'n_estimators': 34, 'max_depth': 90, 'max_leaves': 32, 'learning_rate': 0.8092458606542622, 'booster': 'dart', 'gamma': 27.70865040715498, 'reg_alpha': 9.969288419086604, 'reg_lambda': 9.464427606469147}
Execution time: 1.4123456517855326 minutes


[I 2024-03-08 17:13:03,926] A new study created in memory with name: no-name-dc3f59bc-11bd-498e-ba96-6958ee8a720c
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-08 17:13:05,951] Trial 0 finished with value: 0.584378852708681 and parameters: {'C': 4.89766082982525, 'fit_intercept': True, 'l1_ratio': 0.14283420266111846}. Best is trial 0 with value: 0.584378852708681.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-08 17:13:07,924] Trial 1 finished with value: 0.5818896491483477 and parameters: {'C': 1.4409924612681484,

F1 Score: 0.5958465329109468
Best hyperparameters: {'C': 0.0037193933092718734, 'fit_intercept': True, 'l1_ratio': 0.6367144741072459}
Execution time: 2.7503393292427063 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 17:15:49,913] Trial 0 finished with value: 0.6387197991841858 and parameters: {'C': 0.4115259483013327, 'kernel': 'poly', 'degree': 5, 'gamma': 'auto'}. Best is trial 0 with value: 0.6387197991841858.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 17:16:03,788] Trial 1 finished with va

F1 Score: 0.678879670360265
Best hyperparameters: {'C': 3.3286855698787017, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 2.0}
Execution time: 11.970473432540894 minutes


In [5]:

close_data['xg_boost_buy'] = best_model_xgboost.predict(x)
close_data['reg_log_buy'] = best_model_reg_log.predict(x)
close_data['SVC_buy'] = best_model_SVC.predict(x)
close_data

,Close,X_t-1,X_t-2,X_t-3,RSI,Y_BUY,xg_boost_buy,reg_log_buy,SVC_buy
27,129.413299,129.390106,129.690002,129.910003,28.060866,False,1,False,True
28,129.148498,129.413299,129.390106,129.690002,26.485360,False,1,False,True
29,128.820007,129.148498,129.413299,129.390106,24.701195,False,1,False,True
30,128.580001,128.820007,129.148498,129.413299,23.501640,False,1,False,True
31,128.351699,128.580001,128.820007,129.148498,22.427254,False,1,False,True
...,...,...,...,...,...,...,...,...,...
39574,129.054992,129.039993,129.020004,128.910003,56.509324,False,1,True,True
39575,129.380004,129.054992,129.039993,129.020004,60.700323,False,1,True,True
39576,129.764404,129.380004,129.054992,129.039993,64.854367,False,1,True,True
39577,129.949996,129.764404,129.380004,129.054992,66.620919,False,1,True,True


In [6]:
set(close_data['SVC_buy'])

{True}